In [247]:
import warnings
warnings.filterwarnings('ignore')
!pip3 install ijson

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [248]:
import glob
import ijson
import json
import numpy as np
import os
import pandas as pd
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
import zipfile

In [249]:
DIR_IN = './madb/data/json-ld'
DIR_TMP = './data/preprocess/tmp'
DIR_OUT = './data/preprocess/out'
print(os.getcwd())

/Users/miho/develop/viz-madb_learning


In [250]:
FNS_CM = [
    'cm102',
    'cm105',
    'cm106',
]

In [251]:
# 分析対象とする雑誌名
MCNAMES = [
    '週刊少年ジャンプ',
    '週刊少年マガジン',
    '週刊少年サンデー',
    '週刊少年チャンピオン',
]

In [252]:
COLS_CM105 = [
    'identifier',
    'label',
    'name',
]

In [253]:
# cm102, genre=='雑誌巻号'
COLS_MIS = {
    'identifier': 'miid',
    'label': 'miname',
    'datePublished': 'datePublished',
    'isPartOf': 'mcid',
    'issueNumber': 'issueNumber',
    'numberOfPages': 'numberOfPages',
    'publisher': 'publisher',
    'volumeNumber': 'volumeNumber',
    'price': 'price',
    'editor': 'editor',
}

In [254]:
# cm102, genre=='マンガ作品'
COLS_EPS = {
    'relatedCollection': 'cid',
    'creator': 'creator',
    'note': 'note',
    'alternativeHeadline': 'epname',
    'pageStart': 'pageStart',
    'pageEnd': 'pageEnd',
    'isPartOf': 'miid',
}

In [255]:
# cm106
COLS_CS = {
    'identifier': 'cid',
    'name': 'cname'
}

In [256]:
# 最終的に出力するカラム
COLS_OUT = [
    'mcname', 'miid', 'miname', 'cid', 'cname', 'epname',
    'creator', 'pageStart', 'pageEnd', 'numberOfPages',
    'datePublished', 'price', 'publisher', 'editor',
]

In [257]:
# 許容するpageEnd，pageStartの最大値
MAX_PAGES = 1000

In [258]:
# ここから関数
def read_json(path):
    """
    json file を辞書として読み込む関数
    :param path: read filepath
    :return dict: 辞書
    """
    with open(path, 'r') as f:
        dct = json.load(f)
        return dct

In [259]:
def save_json(path, dct):
    """
    辞書を json 形式で保存する関数
    :param path: json file to save
    :param dct: save dict
    """
    with open(path, 'w') as f:
        json.dump(dct, f, ensure_ascii=False, ident=4)

In [260]:
def read_json_w_filters(path, items, filters):
    """
    item nおうち、filters の条件を満たすもののみを抽出
    :param path: json file path
    :param items:  json file in the item-key
    :param filters: dict, item[key] in value で条件付けする想定
    """
    out = []
    with open(path, 'r') as f:
        parse = ijson.items(f, items)
        for item in parse:
            # filters の条件を全て満足するもの以外は break
            for k, v in filters.items():
                if k not in item.keys():
                    break
                if item[k] not in v:
                    break
            else:
                # break しなかった場合は、 out に追加
                out.append(item)
    return out

In [261]:
def try_mkdirs(path) -> None:
    """mkdirs に try"""
    try:
        os.makedirs(path)
    except FileExistsError as e:
        pass

In [262]:
# 出力フォルダの生成
try_mkdirs(DIR_TMP)
try_mkdirs(DIR_OUT)

In [263]:
# 解凍： マンガ系のデータ (*cm*) のみ DIR_TMP に解凍する
ps_cm = glob.glob(f'{DIR_IN}/*_cm*')

In [264]:
for p_from in tqdm(ps_cm):
    p_to = p_from.replace(DIR_IN, DIR_TMP).replace('.zip', '')

    with zipfile.ZipFile(p_from) as z:
        z.extractall(p_to)

  0%|          | 0/6 [00:00<?, ?it/s]

In [265]:
# 前処理
## 対象
ps_cm = {cm: glob.glob(f'{DIR_TMP}/*{cm}*/*') for cm in FNS_CM}
pprint(ps_cm)

{'cm102': ['./data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00002.json',
           './data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00001.json'],
 'cm105': ['./data/preprocess/tmp/metadata_cm-col_cm105_json/metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json'],
 'cm106': ['./data/preprocess/tmp/metadata_cm-col_cm106_json/metadata_cm-col_cm106_json\\metadata_cm-col_cm106_00001.json']}


In [266]:
# cm105: 漫画雑誌に関するデータを整形し、分析対象のIDを特定する
def format_magazine_name(name):
    """name から published_name を取得"""
    for x in name:
        if type(x) is str:
            return x
    raise Exception(f'No magazine name is {name}!')

In [267]:
cm105 = read_json(ps_cm['cm105'][0])
df_cm105 = pd.DataFrame(cm105['@graph'])[COLS_CM105]

In [268]:
# 雑誌名を取得
df_cm105['mcname'] = df_cm105['name'].apply(lambda x: format_magazine_name(x))

In [269]:
# mcname で抽出
df_cm105[df_cm105['mcname'].isin(MCNAMES)].T

,148,1449,1828,2569
identifier,C117607,C119033,C119459,C120282
label,週刊少年サンデー,週刊少年マガジン,週刊少年ジャンプ,週刊少年チャンピオン
name,"[{'@language': 'ja-hrkt', '@value': 'シュウカンショウネ...","[週刊少年マガジン, {'@language': 'ja-hrkt', '@value': ...","[週刊少年ジャンプ, {'@language': 'ja-hrkt', '@value': ...","[{'@language': 'ja-hrkt', '@value': 'シュウカンショウネ..."
mcname,週刊少年サンデー,週刊少年マガジン,週刊少年ジャンプ,週刊少年チャンピオン


In [270]:
# 雑誌ID: 雑誌名
mcid2mcname = \
    df_cm105[df_cm105['mcname'].isin(MCNAMES)].groupby('identifier')['mcname'].first().to_dict()

In [272]:
print(mcid2mcname)

{'C117607': '週刊少年サンデー', 'C119033': '週刊少年マガジン', 'C119459': '週刊少年ジャンプ', 'C120282': '週刊少年チャンピオン'}


In [284]:
# 保存する
with open(os.path.join(DIR_TMP, 'mcid2mcname.json'), 'w') as f:
  json.dump(mcid2mcname, f, ensure_ascii=False)

In [ ]:
# cm102
# 雑誌巻号および漫画作品に関するデータを整形し、一時保存する
def format_cols(df, cols_rename):
    """cols_rename の col のみを抽出し、rename する"""
    df_new = df.copy()
    df_new = df_new[cols_rename.keys()]
    df_new = df_new.rename(columns=cols_rename)
    return df_new